In [ ]:
pip install torch -U

In [ ]:
import torch
import esm
from model import SparseForTokenClassification
from typing import Tuple, List


### Extracting Embedding for Both Protein Sequences and for Genome
#####  Extracting protien for a single protein 


In [ ]:
_, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()


In [ ]:
# Usign a short 10 amino acid sequence as a proof of concept.

data  = [('seq_1', 'MLKKLSVFLI')]

batch_labels, batch_strs, batch_tokens = batch_converter(data)

##### Recreate the model

In [ ]:
version = '3C' # or '5B'
checkpoint = torch.load(f'./models/{version}/config_and_model.pth', map_location='cpu', weights_only=False)

In [ ]:
device = "cpu"
config = checkpoint['config']
model_state_dict = checkpoint['model_state_dict']
sparse_model = SparseForTokenClassification(config=config)
sparse_model.load_state_dict(model_state_dict)
sparse_model = sparse_model.to(device)
sparse_model = sparse_model.eval()

In [ ]:
def compute_combined_attention(attentions: Tuple[torch.Tensor, ...], weight_scheme: List[float]) -> torch.Tensor:
    x,y,n1,n2 = len(attentions), attentions[0][0].shape[0], attentions[0][0].shape[1], attentions[0][0].shape[2]
    assert len(weight_scheme) == x * y
    
    weighted_sum = torch.zeros_like(attentions[0][0][0])

    stacked_attentions = torch.stack(attentions, dim=0)
    weights = torch.tensor(weight_scheme, dtype=stacked_attentions.dtype, device=stacked_attentions.device)

    flattened_attentions = stacked_attentions.view(x * y, n1, n2)
    weighted_sum = torch.sum(flattened_attentions * weights[:, None, None], dim=0)
    
    return weighted_sum
                     

In [ ]:
output.logits.size()

### We can extract the sequence embeddings using


In [ ]:
output.logits[0,1:-1:].size()

In [ ]:
len(output.attentions)

In [ ]:
[output.attentions[i].size() for i in range(len(output.attentions))]

# Thibaut 

Could you include the code here that combines all the attention heads and layers into a single 10x10 matrix? It should be in a single function. For now, we will provide equal weights to each matrix, but the function should be able to take a different weighting scheme for each layer. See scaffold below


In [ ]:

def compute_combined_attention(attentions: Tuple[torch.Tensor, ...], weight_scheme: List[float]) -> torch.Tensor:
    pass

### Computing the Complete Genome Annotation

In [ ]:
data  = [('seq_1', 'MLKKLSVFLI'), ('seq_2', 'MLSVFLI'), ('seq_3', 'LKKLSVFLI'), ('seq_4', 'MLKKLSVIMMMKV')]

# concat all the sequences in data
prots_concatenated = "".join([x[1] for x in data])
prots_lengths = [len(x[1]) for x in data]
genome = [("some_genome", prots_concatenated)]
genome , prots_lengths

In [ ]:
len(genome[0][1])

In [ ]:
batch_labels, batch_strs, batch_tokens = batch_converter(genome)

In [ ]:
output = sparse_model(input_ids=batch_tokens, 
                      output_attentions = True, 
                      proteins_sizes = torch.tensor(prots_lengths))


In [ ]:
output.keys()

In [ ]:
output['logits'].size()

In [ ]:
len(output['attentions'])

In [ ]:
output['attentions'][0].size()

In [ ]:
sum(prots_lengths)

In [ ]:
# THIBAUT: my understanding is that two_step_selection= True determines whether to return all pairwise scores  or  not.
# Adding here, causes an error.


In [ ]:
output = sparse_model(input_ids=batch_tokens, 
                      output_attentions = True, 
                      proteins_sizes = torch.tensor(prots_lengths),  two_step_selection= True)


### Ranking the pairwise interactions

1. I know that we need to run

In [2]:
from pair_ranking_script import get_pairs_dataset
# I am going to run the code below step by step we can use exactly what is needed
# output = get_pairs_dataset("../data/LR699048.fa", "./data/LR699048.pt", "./data/LR699048.txt", plot_fragment_heatmap = False)
# # generates the folowing
# all_pairs: attention between all pairs of proteins 
# all_nc_pairs:  attention between all pairs of non-consecutive proteins
# global_length: total number of all pairwise comparisons ()
# non_consecutive_length: total number of all non-consecutive pairwise comparisons
# database_dic: same as all_pairs but in dictionary format where key is genome name
# all_top_pairs_labeled: same as all_pairs but with protein labels instead of protein indexes. Uses the labels_file to get the labels
# all_top_pairs_labeled_nc: same as all_nc_pairs but with protein labels instead of protein indexes. Uses the labels_file to get the labels

1 assembled fragments
Number of sequences to process : 0
10 cpus availables
0 fragments processed


In [4]:
from pair_ranking_script import load_fasta_as_tuples
data_assembled = load_fasta_as_tuples("../data/LR699048.fa")
    global_length = []
    non_consecutive_length = []
    all_length_dict = []
    j = 0
    all_top_pairs = []
    all_top_pairs_nc = []
    all_top_pairs_labeled = []
    all_top_pairs_labeled_nc = []
    database_dic = {}
    mlp = []
    genomes = []

In [ ]:
### THIBAUT